<a href="https://colab.research.google.com/github/lastinm/ml_hw_notebooks/blob/main/%D0%A5%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%2C_%D0%B8%D0%B7%D0%B2%D0%BB%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B8_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B8%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%966.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 1. Исходные данные

## 1. Найти (сгенерировать) набор данных, подходящий для выполнения всех заданий.

In [15]:
# импорт необходимых библиотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, min, count, col, mean
import pandas as pd

In [2]:
# Создаем Spark-сессию
spark = SparkSession.builder \
    .appName("Iris Dataset Analysis") \
    .getOrCreate()

## 2. Считать данные в датафрейм.

In [3]:
# Загрузка набора данных Titanic
!wget https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv

--2025-01-07 10:59:21--  https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic.csv.1’

titanic.csv.1       100%[===================>]  58.89K  --.-KB/s    in 0.004s  

2025-01-07 10:59:21 (12.9 MB/s) - ‘titanic.csv.1’ saved [60302/60302]



In [4]:
titanic_df = spark.read.csv("/content/titanic.csv", header=True, inferSchema=True)

# Показать первые 5 строк
titanic_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

## 3. Проверить типы, выполнить преобразования типов, если это требуется.

In [5]:
# Проверка типов данных
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
# Кодирование категориальных признаков. Преобразование текстовых данных в числовые:
from pyspark.ml.feature import StringIndexer

# Кодирование столбца 'Sex'
sex_indexer = StringIndexer(inputCol="Sex", outputCol="Sex_Index")
titanic_df = sex_indexer.fit(titanic_df).transform(titanic_df)

# Кодирование столбца 'Embarked'
embarked_indexer = StringIndexer(inputCol="Embarked", outputCol="Embarked_Index")
titanic_df = embarked_indexer.fit(titanic_df).transform(titanic_df)

In [8]:
# Удаление оригинальных столбцов 'Sex' и 'Embarked'
titanic_df = titanic_df.drop("Sex", "Embarked")

# Теперь вы можете посмотреть результат
titanic_df.show(5)

+-----------+--------+------+--------------------+----+-----+-----+----------------+-------+-----+---------+--------------+
|PassengerId|Survived|Pclass|                Name| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Sex_Index|Embarked_Index|
+-----------+--------+------+--------------------+----+-----+-----+----------------+-------+-----+---------+--------------+
|          1|       0|     3|Braund, Mr. Owen ...|22.0|    1|    0|       A/5 21171|   7.25| NULL|      0.0|           0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|38.0|    1|    0|        PC 17599|71.2833|  C85|      1.0|           1.0|
|          3|       1|     3|Heikkinen, Miss. ...|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|      1.0|           0.0|
|          4|       1|     1|Futrelle, Mrs. Ja...|35.0|    1|    0|          113803|   53.1| C123|      1.0|           0.0|
|          5|       0|     3|Allen, Mr. Willia...|35.0|    0|    0|          373450|   8.05| NULL|      0.0|           0.0|
+-------

## 4. Провести предварительную обработку (все пункты должны быть выполнены):

* Правило для одной колонки: расчет и замена средним по колонке.

In [13]:
# Подсчет количества пропущенных значений в колонке 'Age'
missing_age_count = titanic_df.filter(titanic_df.Age.isNull()).count()

# Общее количество строк
total_count = titanic_df.count()
print(f"Общее количество строк: {total_count}")

# Вывод количества пропущенных значений
print(f"Количество пропущенных значений в колонке 'Age': {missing_age_count}")

Общее количество строк: 891
Количество пропущенных значений в колонке 'Age': 177


In [16]:
# Шаг 1: Рассчитать среднее значение 'Age'
mean_age = titanic_df.select(mean(col("Age"))).first()[0]

# Шаг 2: Заменить пропущенные значения на среднее
titanic_df = titanic_df.na.fill({"Age": mean_age})

# Теперь можно посмотреть обновленные данные
titanic_df.show()

+-----------+--------+------+--------------------+-----------------+-----+-----+----------------+-------+-----+---------+--------------+
|PassengerId|Survived|Pclass|                Name|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Sex_Index|Embarked_Index|
+-----------+--------+------+--------------------+-----------------+-----+-----+----------------+-------+-----+---------+--------------+
|          1|       0|     3|Braund, Mr. Owen ...|             22.0|    1|    0|       A/5 21171|   7.25| NULL|      0.0|           0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|             38.0|    1|    0|        PC 17599|71.2833|  C85|      1.0|           1.0|
|          3|       1|     3|Heikkinen, Miss. ...|             26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|      1.0|           0.0|
|          4|       1|     1|Futrelle, Mrs. Ja...|             35.0|    1|    0|          113803|   53.1| C123|      1.0|           0.0|
|          5|       0|     3|Allen, Mr. W

* Правило для одной колонки: расчет и замена средним по колонке.